## WANDB Scrape and Results Demo

### Imports

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from env.imports import *

In [6]:
import importlib
import data

import models
import sim.sim
import sim.sim_utils
from sim.sim_utils import bytes2human, print_system_usage
from sim.sim import Simulation
from sim.sim_run import single_sim_run
from sim.wandb_scrape import *

Resorting genes by reference genome order


#### Check job specs

In [7]:
print_system_usage()

total = psutil.disk_usage('/').total
print(bytes2human(total))

CPU Usage: 37.5%
RAM Usage: 14.1%
Available RAM: 1.3T
Total RAM: 1.5T
52.4G


## Wandb API Query <a id="sims"></a>

Search and plot Random

In [18]:
from collections import defaultdict

# Define model groups and their valid feature types
models = ['shared_transformer', 'shared_transformer_cls']

# Dictionary mapping models to their required feature types
model_feature_types = {
    'shared_transformer': ['transcriptome'],
    'shared_transformer_cls': ['transcriptome']
}

# Time ranges for true models
time_ranges_true = {
    'default': {'within_last': 100, 'before_last': 0},
    'model_specific': {
        'shared_transformer': {'within_last': 7, 'before_last': -1}, # query one day into future to grab most recent runs
        'shared_transformer_cls': {'within_last': 7, 'before_last': -1},
    }
}

# Time ranges for true models
time_ranges_spin = {
    'default': {'within_last': 100, 'before_last': 0},
    'model_specific': {
        'shared_transformer': {'within_last': 7, 'before_last': -1}, # query one day into future to grab most recent runs
        'shared_transformer_cls': {'within_last': 7, 'before_last': -1},
    }
}

only_include = 'UKBB'
cv_type = 'random'
null_model = 'none'

# Store summary DataFrames
summary_dict_random = {}
process_model_feature_combinations(cv_type, null_model, models, model_feature_types, summary_dict_random, only_include=only_include, time_ranges=time_ranges_true)

cv_type = 'random'
null_model = 'spin'

# Store summary DataFrames
summary_dict_random_spin = {}
process_model_feature_combinations(cv_type, null_model, models, model_feature_types, summary_dict_random_spin,  only_include=only_include, time_ranges=time_ranges_spin)

Checking which model/feature type combinations return 40 runs for null_model=none:

🔍 Fetching runs for: model=shared_transformer, cv_type=random, null_model=none, feature_type=transcriptome
✓ shared_transformer with transcriptome: Successfully found 40 runs
🔍 Fetching runs for: model=shared_transformer_cls, cv_type=random, null_model=none, feature_type=transcriptome
✓ shared_transformer_cls with transcriptome: Successfully found 40 runs
Checking which model/feature type combinations return 40 runs for null_model=spin:

🔍 Fetching runs for: model=shared_transformer, cv_type=random, null_model=spin, feature_type=transcriptome
✓ shared_transformer with transcriptome: Successfully found 40 runs
🔍 Fetching runs for: model=shared_transformer_cls, cv_type=random, null_model=spin, feature_type=transcriptome
✓ shared_transformer_cls with transcriptome: Successfully found 40 runs


In [21]:
summary_dict_random

{'shared_transformer':         train_geodesic_distance  train_inter_hemi_r  \
 mean                  13.090236            0.991317   
 std                    0.950912            0.003502   
 stderr                 0.150352            0.000554   
 
         train_inter_network_Cerebellum_r  train_inter_network_Cont_r  \
 mean                            0.988986                    0.990904   
 std                             0.005343                    0.003599   
 stderr                          0.000845                    0.000569   
 
         train_inter_network_Default_r  train_inter_network_DorsAttn_r  \
 mean                         0.989665                        0.992550   
 std                          0.004368                        0.002821   
 stderr                       0.000691                        0.000446   
 
         train_inter_network_Limbic_r  train_inter_network_SalVentAttn_r  \
 mean                        0.991166                           0.990935   
 std    

In [22]:
summary_dict_random_spin

{'shared_transformer':         train_geodesic_distance  train_inter_hemi_r  \
 mean                  19.713141            0.938344   
 std                    2.414388            0.039721   
 stderr                 0.381748            0.006280   
 
         train_inter_network_Cerebellum_r  train_inter_network_Cont_r  \
 mean                            0.934385                    0.943355   
 std                             0.041803                    0.035954   
 stderr                          0.006610                    0.005685   
 
         train_inter_network_Default_r  train_inter_network_DorsAttn_r  \
 mean                         0.937320                        0.952670   
 std                          0.039408                        0.029880   
 stderr                       0.006231                        0.004724   
 
         train_inter_network_Limbic_r  train_inter_network_SalVentAttn_r  \
 mean                        0.946824                           0.941223   
 std    

Build table with all relevant metrics

In [23]:
summary_dict_random

{'shared_transformer':         train_geodesic_distance  train_inter_hemi_r  \
 mean                  13.090236            0.991317   
 std                    0.950912            0.003502   
 stderr                 0.150352            0.000554   
 
         train_inter_network_Cerebellum_r  train_inter_network_Cont_r  \
 mean                            0.988986                    0.990904   
 std                             0.005343                    0.003599   
 stderr                          0.000845                    0.000569   
 
         train_inter_network_Default_r  train_inter_network_DorsAttn_r  \
 mean                         0.989665                        0.992550   
 std                          0.004368                        0.002821   
 stderr                       0.000691                        0.000446   
 
         train_inter_network_Limbic_r  train_inter_network_SalVentAttn_r  \
 mean                        0.991166                           0.990935   
 std    

In [24]:
def create_test_metric_dataframes(summary_dict):
    # Initialize empty lists for global and subnetwork metrics
    global_metrics = []
    subnetwork_metrics = []
    
    # Define global metric columns
    global_cols = ['pearson_r', 'r2', 'short_r', 'mid_r', 'long_r',
                  'strong_pos_r', 'strong_neg_r', 'weak_r',
                  'left_hemi_r', 'right_hemi_r', 'inter_hemi_r',
                  'mse', 'geodesic_distance']
                  
    # Define subnetworks
    networks = ['Vis', 'SomMot', 'DorsAttn', 'SalVentAttn', 'Limbic', 
                'Cont', 'Default', 'Subcortical', 'Cerebellum']
    
    # Process each model
    model_order = ['shared_transformer_cls', 'dynamic_mlp_coords', 'shared_transformer', 'dynamic_mlp']
    remaining_models = [m for m in reversed(summary_dict.keys()) if m not in model_order]
    ordered_models = model_order + remaining_models
    
    for model in ordered_models:
        if model not in summary_dict:
            continue
            
        df = summary_dict[model]
        
        # Global metrics
        global_row = {}
        for col in global_cols:
            mean = df.loc['mean', f'test_{col}']
            std = df.loc['std', f'test_{col}']
            global_row[col] = f"{mean:.3f} ± {std:.3f}"
        global_metrics.append({'Model': model, **global_row})
        
        # Subnetwork metrics  
        subnetwork_row = {'Model': model}
        for network in networks:
            # Intra-network correlation
            intra_mean = df.loc['mean', f'test_intra_network_{network}_r']
            intra_std = df.loc['std', f'test_intra_network_{network}_r']
            subnetwork_row[f'{network} Intra'] = f"{intra_mean:.3f} ± {intra_std:.3f}"
            
            # Inter-network correlation
            inter_mean = df.loc['mean', f'test_inter_network_{network}_r'] 
            inter_std = df.loc['std', f'test_inter_network_{network}_r']
            subnetwork_row[f'{network} Inter'] = f"{inter_mean:.3f} ± {inter_std:.3f}"
            
        subnetwork_metrics.append(subnetwork_row)
    
    # Create dataframes
    global_df = pd.DataFrame(global_metrics).set_index('Model')
    subnetwork_df = pd.DataFrame(subnetwork_metrics).set_index('Model')
    
    return global_df, subnetwork_df

In [25]:
# Create the dataframes for random
global_metrics_df_random, subnetwork_metrics_df_random = create_test_metric_dataframes(summary_dict_random)

print("\nRandom Test Metrics:")
print("\nGlobal Test Metrics:")
display(global_metrics_df_random)
print("\nSubnetwork Test Metrics:") 
display(subnetwork_metrics_df_random)

# Create the dataframes for random
global_metrics_df_random_spin, subnetwork_metrics_df_random_spin = create_test_metric_dataframes(summary_dict_random_spin)

print("\nRandom Test Spin Metrics:")
print("\nGlobal Test  Spin Metrics:")
display(global_metrics_df_random_spin)


Random Test Metrics:

Global Test Metrics:


,pearson_r,r2,short_r,mid_r,long_r,strong_pos_r,strong_neg_r,weak_r,left_hemi_r,right_hemi_r,inter_hemi_r,mse,geodesic_distance
Model,,,,,,,,,,,,,
shared_transformer_cls,0.844 ± 0.022,0.707 ± 0.040,0.862 ± 0.020,0.820 ± 0.027,0.783 ± 0.057,0.550 ± 0.059,0.250 ± 0.126,0.750 ± 0.033,0.829 ± 0.034,0.863 ± 0.026,0.842 ± 0.024,0.010 ± 0.001,9.803 ± 0.928
shared_transformer,0.795 ± 0.028,0.613 ± 0.054,0.826 ± 0.025,0.764 ± 0.032,0.691 ± 0.073,0.479 ± 0.071,0.142 ± 0.129,0.688 ± 0.033,0.779 ± 0.042,0.833 ± 0.031,0.787 ± 0.032,0.013 ± 0.001,10.284 ± 1.028



Subnetwork Test Metrics:


,Vis Intra,Vis Inter,SomMot Intra,SomMot Inter,DorsAttn Intra,DorsAttn Inter,SalVentAttn Intra,SalVentAttn Inter,Limbic Intra,Limbic Inter,Cont Intra,Cont Inter,Default Intra,Default Inter,Subcortical Intra,Subcortical Inter,Cerebellum Intra,Cerebellum Inter
Model,,,,,,,,,,,,,,,,,,
shared_transformer_cls,0.886 ± 0.059,0.872 ± 0.024,0.876 ± 0.061,0.875 ± 0.029,0.861 ± 0.076,0.876 ± 0.028,0.853 ± 0.093,0.867 ± 0.028,0.856 ± 0.144,0.867 ± 0.031,0.838 ± 0.102,0.853 ± 0.040,0.809 ± 0.081,0.843 ± 0.034,0.789 ± 0.112,0.855 ± 0.028,0.684 ± 0.507,0.853 ± 0.035
shared_transformer,0.871 ± 0.054,0.843 ± 0.023,0.842 ± 0.074,0.846 ± 0.031,0.825 ± 0.134,0.845 ± 0.035,0.815 ± 0.143,0.833 ± 0.040,0.846 ± 0.116,0.834 ± 0.041,0.787 ± 0.107,0.814 ± 0.042,0.795 ± 0.056,0.812 ± 0.029,0.665 ± 0.211,0.808 ± 0.033,0.750 ± 0.338,0.807 ± 0.043



Random Test Spin Metrics:

Global Test  Spin Metrics:


,pearson_r,r2,short_r,mid_r,long_r,strong_pos_r,strong_neg_r,weak_r,left_hemi_r,right_hemi_r,inter_hemi_r,mse,geodesic_distance
Model,,,,,,,,,,,,,
shared_transformer_cls,0.732 ± 0.050,0.502 ± 0.089,0.759 ± 0.051,0.687 ± 0.057,0.666 ± 0.079,0.460 ± 0.075,0.164 ± 0.126,0.608 ± 0.060,0.705 ± 0.074,0.757 ± 0.058,0.731 ± 0.054,0.017 ± 0.003,11.611 ± 0.985
shared_transformer,0.343 ± 0.086,-0.115 ± 0.149,0.344 ± 0.089,0.288 ± 0.085,0.280 ± 0.113,0.174 ± 0.071,0.048 ± 0.131,0.252 ± 0.071,0.341 ± 0.100,0.355 ± 0.096,0.340 ± 0.087,0.038 ± 0.006,15.223 ± 1.430
